# Clase 7: Taller Práctico - Otros Métodos Supervisados

**Objetivos:**

En este taller, aplicaremos y compararemos los tres métodos de clasificación que hemos estudiado en la clase teórica:

1.  **k-Nearest Neighbors (k-NN)**: Un clasificador basado en instancia y distancia.
2.  **Naive Bayes**: Un clasificador probabilístico basado en el teorema de Bayes.
3.  **Perceptrón Multicapa (MLP)**: Nuestro primer vistazo a una red neuronal simple.

Utilizaremos el conjunto de datos "Wine" de Scikit-learn, un dataset clásico para problemas de clasificación multiclase. El objetivo es clasificar vinos en una de tres categorías basándonos en sus atributos químicos.

## 1. Configuración Inicial e Importación de Librerías

In [91]:
# Librerías para manipulación de datos
import pandas as pd
import numpy as np

import time

# Librerías de Scikit-learn para el modelado
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.decomposition import PCA

# Librerías para visualización
import plotly.express as px
import plotly.graph_objects as go

## 2. Carga y Exploración del Dataset (EDA)

El dataset "Wine" contiene los resultados de un análisis químico de vinos cultivados en la misma región en Italia pero derivados de tres cultivares diferentes. El análisis determinó las cantidades de 13 constituyentes encontrados en cada uno de los tres tipos de vinos.

In [92]:
# Cargar el dataset
wine_data = load_wine()
X = pd.DataFrame(wine_data.data, columns=wine_data.feature_names)
y = pd.Series(wine_data.target, name='target')

# Unir características y objetivo en un solo DataFrame para exploración
df = pd.concat([X, y], axis=1)

# Mapear los números del objetivo a nombres de clases para mayor claridad
df['target_name'] = df['target'].map({0: 'Class_0', 1: 'Class_1', 2: 'Class_2'})

print("Primeras 5 filas del dataset:")
display(df.head())

print("\nDescripción estadística de las características:")
display(df.describe())

Primeras 5 filas del dataset:


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target,target_name
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0,Class_0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0,Class_0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0,Class_0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0,Class_0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0,Class_0



Descripción estadística de las características:


,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,target
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258,0.938202
std,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474,0.775035
min,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000,0.000000
25%,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000,0.000000
50%,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000,1.000000
75%,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000,2.000000
max,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000,2.000000


### Visualización con PCA

Dado que tenemos 13 características, no podemos visualizarlas todas a la vez. Usaremos el Análisis de Componentes Principales (PCA) para reducir la dimensionalidad a 2 componentes y visualizar la separación de las clases.

In [93]:
# Primero, escalamos los datos para que PCA funcione correctamente
scaler_pca = StandardScaler()
X_scaled_pca = scaler_pca.fit_transform(X)

# Aplicamos PCA para reducir a 2 dimensiones
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled_pca)

# Creamos un DataFrame para la visualización
df_pca = pd.DataFrame(data=X_pca, columns=['PC1', 'PC2'])
df_pca['target_name'] = df['target_name']

# Gráfico interactivo con Plotly
fig = px.scatter(df_pca, 
                 x='PC1', 
                 y='PC2', 
                 color='target_name', 
                 title='Visualización del Dataset Wine con PCA (2 Componentes)',
                 labels={'PC1': 'Primer Componente Principal', 'PC2': 'Segundo Componente Principal'},
                 template='plotly_white')

fig.show()

La visualización PCA nos muestra que las clases están razonablemente bien separadas, aunque con algo de superposición. Esto sugiere que los algoritmos de clasificación deberían poder encontrar patrones útiles.

## 3. Preparación de los Datos para el Modelado

Ahora, dividiremos los datos en conjuntos de entrenamiento y prueba, y aplicaremos el escalado de características. El escalado es fundamental para k-NN y MLP, ya que son sensibles a las diferentes escalas de las variables.

In [94]:
# Separar los datos originales (sin PCA) en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Inicializar el escalador
scaler = StandardScaler()

# Ajustar el escalador SÓLO con los datos de entrenamiento
X_train_scaled = scaler.fit_transform(X_train)

# Aplicar la misma transformación a los datos de prueba
X_test_scaled = scaler.transform(X_test)

print(f"Tamaño del set de entrenamiento: {X_train_scaled.shape[0]} muestras")
print(f"Tamaño del set de prueba: {X_test_scaled.shape[0]} muestras")

Tamaño del set de entrenamiento: 124 muestras
Tamaño del set de prueba: 54 muestras


## 4. Implementación y Evaluación de Modelos

### Modelo 1: k-Nearest Neighbors (k-NN)

Implementaremos k-NN con un valor de `k` inicial (ej. k=5) y evaluaremos su rendimiento.

In [95]:
start = time.time()
# Inicializar y entrenar el clasificador k-NN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_knn = knn.predict(X_test_scaled)

# Evaluar el modelo
print("------ Resultados de k-NN (k=5) ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_knn):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_knn))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_knn))
end = time.time()
print(f"{end - start:.3f} segundos para entrenar el modelo k-NN")

------ Resultados de k-NN (k=5) ------
Accuracy: 0.9444

Matriz de Confusión:
[[18  0  0]
 [ 0 18  3]
 [ 0  0 15]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      0.86      0.92        21
           2       0.83      1.00      0.91        15

    accuracy                           0.94        54
   macro avg       0.94      0.95      0.94        54
weighted avg       0.95      0.94      0.94        54

0.030 segundos para entrenar el modelo k-NN


#### Desafío: Encontrar el `k` Óptimo

El rendimiento de k-NN depende fuertemente del valor de `k`. Un `k` muy pequeño puede llevar a sobreajuste, mientras que un `k` muy grande puede simplificar demasiado el modelo. 

**Tu tarea:** Escribe un bucle que entrene y evalúe el modelo k-NN para un rango de valores de `k` (por ejemplo, de 1 a 30). Almacena la precisión (accuracy) para cada `k` y luego crea un gráfico para visualizar cómo cambia la precisión en función de `k`. ¿Cuál es el valor de `k` que da el mejor resultado en el conjunto de prueba?

In [96]:
# Espacio para tu solución al desafío
k_values = range(1, 31)
accuracies = []

for k in k_values:
    knn_loop = KNeighborsClassifier(n_neighbors=k)
    knn_loop.fit(X_train_scaled, y_train)
    y_pred_loop = knn_loop.predict(X_test_scaled)
    accuracies.append(accuracy_score(y_test, y_pred_loop))

# Gráfico de k vs. Accuracy
fig = go.Figure(data=go.Scatter(x=list(k_values), y=accuracies, mode='lines+markers'))
fig.update_layout(
    title='Accuracy de k-NN en función del número de vecinos (k)',
    xaxis_title='Valor de k',
    yaxis_title='Accuracy en el conjunto de prueba',
    template='plotly_white'
)
fig.show()

best_k = k_values[np.argmax(accuracies)]
print(f"\nEl valor óptimo de k es: {best_k} con una precisión de {max(accuracies):.4f}")


El valor óptimo de k es: 21 con una precisión de 1.0000


### Modelo 2: Naive Bayes

Ahora, aplicaremos el clasificador Naive Bayes Gaussiano. Esta variante es adecuada porque nuestras características son continuas y podemos asumir (ingenuamente) que siguen una distribución normal dentro de cada clase.

In [97]:
start = time.time()
# Inicializar y entrenar el clasificador Naive Bayes Gaussiano
# Nota: Naive Bayes no es tan sensible al escalado, pero es buena práctica usar los datos escalados por consistencia.
gnb = GaussianNB()
gnb.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_gnb = gnb.predict(X_test_scaled)

# Evaluar el modelo
print("------ Resultados de Naive Bayes Gaussiano ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_gnb):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_gnb))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_gnb))
end = time.time()
print(f"{end - start:.3f} segundos para entrenar el modelo Naive Bayes")

------ Resultados de Naive Bayes Gaussiano ------
Accuracy: 1.0000

Matriz de Confusión:
[[18  0  0]
 [ 0 21  0]
 [ 0  0 15]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00        15

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54

0.027 segundos para entrenar el modelo Naive Bayes


### Modelo 3: Perceptrón Multicapa (MLP)

Finalmente, implementaremos una red neuronal simple. Usaremos `MLPClassifier` de Scikit-learn, que es una implementación eficiente y fácil de usar. Definiremos una arquitectura simple con una capa oculta.

In [98]:
start = time.time()
# Inicializar y entrenar el clasificador MLP
# hidden_layer_sizes=(100,) significa una capa oculta con 100 neuronas.
# max_iter=1000 para asegurar la convergencia.
mlp = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
mlp.fit(X_train_scaled, y_train)

# Realizar predicciones
y_pred_mlp = mlp.predict(X_test_scaled)

# Evaluar el modelo
print("------ Resultados del Perceptrón Multicapa (MLP) ------")
print(f"Accuracy: {accuracy_score(y_test, y_pred_mlp):.4f}")
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, y_pred_mlp))
print("\nReporte de Clasificación:")
print(classification_report(y_test, y_pred_mlp))
end = time.time()
print(f"{end - start:.3f} segundos para entrenar el modelo MLP")

------ Resultados del Perceptrón Multicapa (MLP) ------
Accuracy: 1.0000

Matriz de Confusión:
[[18  0  0]
 [ 0 21  0]
 [ 0  0 15]]

Reporte de Clasificación:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        18
           1       1.00      1.00      1.00        21
           2       1.00      1.00      1.00        15

    accuracy                           1.00        54
   macro avg       1.00      1.00      1.00        54
weighted avg       1.00      1.00      1.00        54

0.198 segundos para entrenar el modelo MLP


## 5. Comparación y Conclusiones

En este caso particular, los tres modelos han obtenido una precisión muy alta, a menudo perfecta o casi perfecta en el conjunto de prueba. Esto se debe a que el dataset Wine es un problema relativamente "fácil" con clases bien separadas.

* **k-NN** demostró ser muy efectivo, especialmente después de encontrar un buen valor para `k`.
* **Naive Bayes** también funcionó excepcionalmente bien, lo que sugiere que, para este problema, el supuesto de independencia condicional no fue una limitación grave.
* El **MLP** igualó el rendimiento de los otros modelos, mostrando su capacidad para resolver problemas de clasificación, aunque su entrenamiento es computacionalmente más costoso.

En problemas del mundo real más complejos y con mayor superposición entre clases, las diferencias en el rendimiento de estos algoritmos suelen ser más pronunciadas. La elección del modelo dependerá de las características específicas del problema, la cantidad de datos disponibles y los recursos computacionales.

## 6. Ejercicios Adicionales

Para solidificar tu comprensión, aquí tienes 10 ejercicios para explorar por tu cuenta.

1.  **Cambiar el Dataset:** Carga el dataset `load_breast_cancer` de `sklearn.datasets`. Es un problema de clasificación binaria. Repite el análisis completo (EDA, preprocesamiento, modelado y comparación) para este nuevo dataset. ¿Qué modelo funciona mejor aquí?

2. **Arquitectura del MLP:** Experimenta con diferentes arquitecturas para el `MLPClassifier`. Prueba con más capas ocultas (ej. `hidden_layer_sizes=(100, 50,)`) o con diferente número de neuronas por capa. ¿Puedes mejorar la precisión obtenida?

3. **Métricas de Distancia en k-NN:** El `KNeighborsClassifier` tiene un parámetro `metric`. Por defecto es `'minkowski'` (que con p=2 es la distancia Euclidiana). Prueba a cambiarlo a `'manhattan'` (Distancia de Manhattan). ¿Cambia el rendimiento del modelo?

4. **Robustez del `train_test_split`:** Cambia el valor de `random_state` en la función `train_test_split`. Repite el entrenamiento y evaluación de los tres modelos. ¿Son los resultados de precisión exactamente los mismos? ¿Qué nos dice esto sobre la evaluación de modelos?

5. **Comparar con Regresión Logística:** Como un modelo de base adicional, importa `LogisticRegression` de `sklearn.linear_model`. Entrénalo y evalúalo en el dataset Wine. ¿Cómo se compara su rendimiento con los tres modelos de esta clase?

---------------------

#### **1. Cambiar el Dataset:** Carga el dataset `load_breast_cancer` de `sklearn.datasets`. Es un problema de clasificación binaria. Repite el análisis completo (EDA, preprocesamiento, modelado y comparación) para este nuevo dataset. ¿Qué modelo funciona mejor aquí?

`Respuesta`:

In [99]:
# ======================================================================================================================================
#
# PREPARACIÓN DEL ENTORNO
#
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

# ======================================================================================================================================
#
# CARGA Y EXPLORACIÓN DE DATOS
#
from sklearn.datasets import load_breast_cancer

# Carga
data_cancer = load_breast_cancer(as_frame=True)
df_cancer = data_cancer.frame

# Características y objetivo
X_cancer = df_cancer[data_cancer.feature_names]
y_cancer = df_cancer['target']

# Clases
display(Markdown(f"**Nombres de clases**: {data_cancer.target_names}"))

# Primeras filas del DF
display(Markdown("\n**Primeras 5 filas del dataset de cáncer de mama**:"))
display(df_cancer.head())

# Resumen estadístico del DF
display(Markdown("\n**Resumen estadístico del dataset de cáncer de mama**:"))
display(df_cancer.describe())

# ======================================================================================================================================
#
# VISUALIZACIÓN CON PCA
#

# Escalado de datos, necesario para PCA
scaler_pca_cancer = StandardScaler()
X_scaled_pca_cancer = scaler_pca_cancer.fit_transform(X_cancer)

# Aplicamos PCA para reducir a 2 dimensiones
pca_cancer = PCA(n_components=2)
X_pca_cancer = pca_cancer.fit_transform(X_scaled_pca_cancer)

# Creamos un DataFrame para la visualización
df_pca_cancer = pd.DataFrame(data=X_pca_cancer, columns=['PC1', 'PC2'])
df_pca_cancer['target_name'] = y_cancer.map({0: 'Maligno', 1: 'Benigno'})

# Gráfico interactivo con Plotly
fig_cancer = px.scatter(df_pca_cancer,
                        x='PC1', 
                        y='PC2', 
                        color='target_name', 
                        title='Visualización del Dataset Cáncer de Mama con PCA (2 Componentes)',
                        labels={'PC1': 'Primer Componente Principal', 'PC2': 'Segundo Componente Principal'},
                        template='plotly_white')
fig_cancer.show()

# ======================================================================================================================================
#
# PREPARACIÓN DE DATOS
#

# Separación
X_train_cancer, X_test_cancer, y_train_cancer, y_test_cancer = train_test_split(X_cancer, y_cancer, test_size=0.2, random_state=42, stratify=y_cancer)

# Nuevo escalado, en este caso, para el entrenamiento y prueba
scaler_cancer = StandardScaler()
X_train_scaled_cancer = scaler_cancer.fit_transform(X_train_cancer)
X_test_scaled_cancer = scaler_cancer.transform(X_test_cancer)

# Visualización de sets
display(Markdown(f"Tamaño del set de entrenamiento: {X_train_scaled_cancer.shape[0]}"))
display(Markdown(f"Tamaño del set de prueba: {X_test_scaled_cancer.shape[0]}"))

# ======================================================================================================================================
#
# MODELADO CON K-NN
#

start_cancer_knn = time.time()

# Inicializar y entrenar el clasificador k-NN
knn_cancer = KNeighborsClassifier(n_neighbors=5)
knn_cancer.fit(X_train_scaled_cancer, y_train_cancer)

# Realizar predicciones
y_pred_cancer_knn = knn_cancer.predict(X_test_scaled_cancer)

# Evaluar el modelo
display(Markdown("------ Resultados de k-NN (k=5) para Cáncer de Mama ------"))
display(Markdown(f"**Accuracy**: {accuracy_score(y_test_cancer, y_pred_cancer_knn):.4f}"))
display(Markdown("\n**Matriz de Confusión:**"))
print(confusion_matrix(y_test_cancer, y_pred_cancer_knn))
display(Markdown("\n**Reporte de Clasificación:**"))
print(classification_report(y_test_cancer, y_pred_cancer_knn))
end_cancer_knn = time.time()
display(Markdown(f"**{end_cancer_knn - start_cancer_knn:.3f}** segundos para entrenar el modelo k-NN para Cáncer de Mama"))
display(Markdown("\n------------------------------------------------------------\n"))

# ======================================================================================================================================
#
# MODELADO CON NAIVE BAYES
#

start_cancer_gnb = time.time()

# Inicializar y entrenar el clasificador Naive Bayes
gnb_cancer = GaussianNB()
gnb_cancer.fit(X_train_scaled_cancer, y_train_cancer)

# Realizar predicciones
y_pred_cancer_gnb = gnb_cancer.predict(X_test_scaled_cancer)

# Evaluar el modelo
display(Markdown("------ Resultados de Naive Bayes Gaussiano para Cáncer de Mama ------"))
display(Markdown(f"**Accuracy**: {accuracy_score(y_test_cancer, y_pred_cancer_gnb):.4f}"))
display(Markdown("\n**Matriz de Confusión:**"))
print(confusion_matrix(y_test_cancer, y_pred_cancer_gnb))
display(Markdown("\n**Reporte de Clasificación:**"))
print(classification_report(y_test_cancer, y_pred_cancer_gnb))
end_cancer_gnb = time.time()
display(Markdown(f"**{end_cancer_gnb - start_cancer_gnb:.3f}** segundos para entrenar el modelo Naive Bayes para Cáncer de Mama"))
display(Markdown("\n------------------------------------------------------------\n"))

# ======================================================================================================================================
#
# MODELADO CON MLP
#

start_cancer_mlp = time.time()

# Inicializar y entrenar el clasificador MLP
mlp_cancer = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
mlp_cancer.fit(X_train_scaled_cancer, y_train_cancer)

# Realizar predicciones
y_pred_cancer_mlp = mlp_cancer.predict(X_test_scaled_cancer)

# Evaluar el modelo
display(Markdown("------ Resultados del Perceptrón Multicapa (MLP) para Cáncer de Mama ------"))
display(Markdown(f"**Accuracy**: {accuracy_score(y_test_cancer, y_pred_cancer_mlp):.4f}"))
display(Markdown("\n**Matriz de Confusión:**"))
print(confusion_matrix(y_test_cancer, y_pred_cancer_mlp))
display(Markdown("\n**Reporte de Clasificación:**"))
print(classification_report(y_test_cancer, y_pred_cancer_mlp))
end_cancer_mlp = time.time()
display(Markdown(f"**{end_cancer_mlp - start_cancer_mlp:.3f}** segundos para entrenar el modelo MLP para Cáncer de Mama"))
display(Markdown("\n------------------------------------------------------------\n"))

# ======================================================================================================================================
#
# COMPARACIÓN DE MODELOS
#

display(Markdown("### Comparación de Modelos para el Dataset Cáncer de Mama"))
display(Markdown("#### **Precisión**"))
display(Markdown("Según `accurancy`, el modelo que mejor funciona para el dataset de cáncer de mama, es Preceptrón Multicapa (MLP) con un accuracy de 0.9649, seguido por k-NN con 0.9561 y Naive Bayes con 0.9298."))
display(Markdown("\n"))
display(Markdown("#### **Matriz de confusión**"))
display(Markdown("Si miramos la `matriz de confusión`, no hay tanta diferencia ente los modelos. Pero teniendo en cuenta las características del dataset, podemos considerar que el costo de los falsos negativos tiene mayor peso. Y el modelo que minimiza los falsos negativos es K-NN, con 1 falso negativo."))
display(Markdown("\n"))
display(Markdown("#### **Tiempo de entrenamiento**"))
display(Markdown("En cuanto al `tiempo de entrenamiento`, el modelo más rápido es Naive Bayes, seguido por k-NN y MLP. Pero, la diferencia no es tan significativa entre ellos."))
display(Markdown("\n"))

**Nombres de clases**: ['malignant' 'benign']


**Primeras 5 filas del dataset de cáncer de mama**:

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0



**Resumen estadístico del dataset de cáncer de mama**:

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946,0.627417
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061,0.483918
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040,0.000000
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460,0.000000
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040,1.000000
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080,1.000000
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500,1.000000


...

Según `accurancy`, el modelo que mejor funciona para el dataset de cáncer de mama, es Preceptrón Multicapa (MLP) con un accuracy de 0.9649, seguido por k-NN con 0.9561 y Naive Bayes con 0.9298.

#### **Matriz de confusión**

Si miramos la `matriz de confusión`, no hay tanta diferencia ente los modelos. Pero teniendo en cuenta las características del dataset, podemos considerar que el costo de los falsos negativos tiene mayor peso. Y el modelo que minimiza los falsos negativos es K-NN, con 1 falso negativo.

#### **Tiempo de entrenamiento**

En cuanto al `tiempo de entrenamiento`, el modelo más rápido es Naive Bayes, seguido por k-NN y MLP. Pero, la diferencia no es tan significativa entre ellos.

#### 2. **Arquitectura del MLP:** Experimenta con diferentes arquitecturas para el `MLPClassifier`. Prueba con más capas ocultas (ej. `hidden_layer_sizes=(100, 50,)`) o con diferente número de neuronas por capa. ¿Puedes mejorar la precisión obtenida?

`Respuesta`:

In [100]:
# ======================================================================================================================================
#
# PREPARACIÓN DEL ENTORNO
#
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

# ======================================================================================================================================
#
# CARGA Y EXPLORACIÓN DE DATOS
#
from sklearn.datasets import load_breast_cancer

# Carga
data_cancer_2 = load_breast_cancer(as_frame=True)
df_cancer_2 = data_cancer_2.frame

# Características y objetivo
X_cancer_2 = df_cancer_2[data_cancer_2.feature_names]
y_cancer_2 = df_cancer_2['target']

# ======================================================================================================================================
#
# PREPARACIÓN DE DATOS
#

# Separación
X_train_cancer_2, X_test_cancer_2, y_train_cancer_2, y_test_cancer_2 = train_test_split(X_cancer_2, y_cancer_2, test_size=0.2, random_state=42, stratify=y_cancer_2)

# Nuevo escalado, en este caso, para el entrenamiento y prueba
scaler_cancer_2 = StandardScaler()
X_train_scaled_cancer_2 = scaler_cancer_2.fit_transform(X_train_cancer_2)
X_test_scaled_cancer_2 = scaler_cancer_2.transform(X_test_cancer_2)

# ======================================================================================================================================
#
# MODELADO CON MLP
#

# Defino las vbles sobre las cuales voy a basar la iteración del entrenamiento
hidden_layers_sizes_2 = [
    (100,),
    (100, 50),
    (100, 50, 25),
    (200,),
    (200, 100),
    (200, 100, 50,),
    (200, 100, 50, 25),
    (300,),
    (300, 200),
    (300, 200, 100),
    (300, 200, 100, 50),
    (300, 200, 100, 50, 25),
]
corridas = []
display(Markdown(f"Entrenamiento MLP para diferentes combinaciones de capas ocultas y neuronas"))
for layer_size in hidden_layers_sizes_2:
    # Inicializar y entrenar el clasificador MLP
    mlp_cancer_2 = MLPClassifier(layer_size, max_iter=1000, random_state=42)
    mlp_cancer_2.fit(X_train_scaled_cancer_2, y_train_cancer_2)
    # Realizar predicciones
    y_pred_cancer_mlp_2 = mlp_cancer_2.predict(X_test_scaled_cancer_2)
    # Evaluar la precisión del modelo
    accuracy = accuracy_score(y_test_cancer_2, y_pred_cancer_mlp_2)
    display(Markdown(f"------> **Accuracy** para {len(layer_size)} capas {layer_size}: {accuracy:.4f} "))
    corridas.append({
        'hidden_layer_sizes': layer_size,
        'accuracy': accuracy
    })
display(Markdown("`CONCLUSIONES`:"))
display(Markdown(f"Se corre el entrenamiento, para distintas cantidades de capas ocultas. No se observan cambios significativos en la precisión, por lo que podemos concluir (al menos con las corridas realizadas para este ejericio), que más capas no mejoran la precisión del modelo."))



Entrenamiento MLP para diferentes combinaciones de capas ocultas y neuronas

------> **Accuracy** para 1 capas (100,): 0.9649 

------> **Accuracy** para 2 capas (100, 50): 0.9474 

------> **Accuracy** para 3 capas (100, 50, 25): 0.9474 

------> **Accuracy** para 1 capas (200,): 0.9649 

...

------> **Accuracy** para 3 capas (300, 200, 100): 0.9649 

------> **Accuracy** para 4 capas (300, 200, 100, 50): 0.9649 

------> **Accuracy** para 5 capas (300, 200, 100, 50, 25): 0.9649 

`CONCLUSIONES`:

Se corre el entrenamiento, para distintas cantidades de capas ocultas. No se observan cambios significativos en la precisión, por lo que podemos concluir (al menos con las corridas realizadas para este ejericio), que más capas no mejoran la precisión del modelo.

#### 3. **Métricas de Distancia en k-NN:** El `KNeighborsClassifier` tiene un parámetro `metric`. Por defecto es `'minkowski'` (que con p=2 es la distancia Euclidiana). Prueba a cambiarlo a `'manhattan'` (Distancia de Manhattan). ¿Cambia el rendimiento del modelo?

`Respuesta`:

In [101]:
# ======================================================================================================================================
#
# PREPARACIÓN DEL ENTORNO
#
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

# ======================================================================================================================================
#
# CARGA Y EXPLORACIÓN DE DATOS
#
from sklearn.datasets import load_breast_cancer

# Carga
data_cancer_3 = load_breast_cancer(as_frame=True)
df_cancer_3 = data_cancer_3.frame

# Características y objetivo
X_cancer_3 = df_cancer_3[data_cancer_3.feature_names]
y_cancer_3 = df_cancer_3['target']

# ======================================================================================================================================
#
# PREPARACIÓN DE DATOS
#

# Separación
X_train_cancer_3, X_test_cancer_3, y_train_cancer_3, y_test_cancer_3 = train_test_split(X_cancer_3, y_cancer_3, test_size=0.2, random_state=42, stratify=y_cancer_3)

# Nuevo escalado, en este caso, para el entrenamiento y prueba
scaler_cancer_3 = StandardScaler()
X_train_scaled_cancer_3 = scaler_cancer_3.fit_transform(X_train_cancer_3)
X_test_scaled_cancer_3 = scaler_cancer_3.transform(X_test_cancer_3)

# ======================================================================================================================================
#
# MODELADO CON K-NN
#

# Inicializar y entrenar el clasificador k-NN
knn_cancer_3 = KNeighborsClassifier(n_neighbors=5, metric='manhattan')
knn_cancer_3.fit(X_train_scaled_cancer_3, y_train_cancer_3)

# Realizar predicciones
y_pred_cancer_knn_3 = knn_cancer_3.predict(X_test_scaled_cancer_3)

# Evaluar el modelo
display(Markdown("------ Resultados de k-NN (k=5) para Cáncer de Mama ------"))
display(Markdown(f"**Accuracy k-NN (manhattan)**: {accuracy_score(y_test_cancer_3, y_pred_cancer_knn_3):.4f}"))
display(Markdown(f"**Accuracy k-NN (euclidiana)**: {accuracy_score(y_test_cancer, y_pred_cancer_knn):.4f}"))

display(Markdown("\n**Matriz de Confusión (manhattan):**"))
print(confusion_matrix(y_test_cancer_3, y_pred_cancer_knn_3))
display(Markdown("\n**Matriz de Confusión (euclidiana):**"))
print(confusion_matrix(y_test_cancer, y_pred_cancer_knn))

display(Markdown("\n**Reporte de Clasificación (manhattan):**"))
print(classification_report(y_test_cancer_3, y_pred_cancer_knn_3))
display(Markdown("\n**Reporte de Clasificación (euclidiana):**"))
print(classification_report(y_test_cancer, y_pred_cancer_knn))

# ======================================================================================================================================
#
# COMPARACIÓN DE ENTRENAMIENTOS PARA k-NN
#

display(Markdown("### Comparación de entrenamientos k-NN (`manhattan` vs `euclidiana`)"))
display(Markdown("#### **Precisión**"))
display(Markdown("Según `accurancy`, **manhattan** funciona mejor (0.9649 vs 0.9561)."))
display(Markdown("---"))
display(Markdown("#### **Matriz de confusión**"))
display(Markdown("Si miramos la `matriz de confusión`, podemos concluir que otra vez **manhattan** funciona mejor, dado que minimiza los falsos negativos (1 vs 2)."))
display(Markdown("---"))
display(Markdown("#### **Reporte de clasificación**"))
display(Markdown("Mirando el `Reporte de clasificación`, una vez más **manhattan** parece ser mejor. Para la clase `maligno`, el recall (pocos falsos negativos) mejora (0.99 vs 0.97). "))

------ Resultados de k-NN (k=5) para Cáncer de Mama ------

**Accuracy k-NN (manhattan)**: 0.9649

**Accuracy k-NN (euclidiana)**: 0.9561


**Matriz de Confusión (manhattan):**

[[39  3]
 [ 1 71]]


...

#### **Matriz de confusión**

Si miramos la `matriz de confusión`, podemos concluir que otra vez **manhattan** funciona mejor, dado que minimiza los falsos negativos (1 vs 2).

---

#### **Reporte de clasificación**

Mirando el `Reporte de clasificación`, una vez más **manhattan** parece ser mejor. Para la clase `maligno`, el recall (pocos falsos negativos) mejora (0.99 vs 0.97). 

#### 4. **Robustez del `train_test_split`:** Cambia el valor de `random_state` en la función `train_test_split`. Repite el entrenamiento y evaluación de los tres modelos. ¿Son los resultados de precisión exactamente los mismos? ¿Qué nos dice esto sobre la evaluación de modelos?

`Respuesta`:

In [102]:
# ======================================================================================================================================
#
# PREPARACIÓN DEL ENTORNO
#
import pandas as pd
import numpy as np
from IPython.display import display, Markdown

# ======================================================================================================================================
#
# CARGA Y EXPLORACIÓN DE DATOS
#
from sklearn.datasets import load_breast_cancer

# Carga
data_cancer_4 = load_breast_cancer(as_frame=True)
df_cancer_4 = data_cancer_4.frame

# Características y objetivo
X_cancer_4 = df_cancer_4[data_cancer_4.feature_names]
y_cancer_4 = df_cancer_4['target']

# ======================================================================================================================================
#
# PREPARACIÓN DE DATOS
#

ramdom_states = [42, 84, 168, 336, 672, 1344]

for random_state in ramdom_states:
    # Separación
    X_train_cancer_4, X_test_cancer_4, y_train_cancer_4, y_test_cancer_4 = train_test_split(X_cancer_4, y_cancer_4, test_size=0.2, random_state=random_state, stratify=y_cancer_4)

    # Nuevo escalado, en este caso, para el entrenamiento y prueba
    scaler_cancer_4 = StandardScaler()
    X_train_scaled_cancer_4 = scaler_cancer_4.fit_transform(X_train_cancer_4)
    X_test_scaled_cancer_4 = scaler_cancer_4.transform(X_test_cancer_4)

    # ======================================================================================================================================
    #
    # MODELADO CON K-NN
    #

    # Inicializar y entrenar el clasificador k-NN
    knn_cancer_4 = KNeighborsClassifier(n_neighbors=5)
    knn_cancer_4.fit(X_train_scaled_cancer_4, y_train_cancer_4)

    # Realizar predicciones
    y_pred_cancer_knn_4 = knn_cancer_4.predict(X_test_scaled_cancer_4)

    # Evaluar el modelo
    display(Markdown(f"------> **Accuracy K-NN ({random_state})**: {accuracy_score(y_test_cancer_4, y_pred_cancer_knn_4):.4f}"))

    # ======================================================================================================================================
    #
    # MODELADO CON NAIVE BAYES
    #

    # Inicializar y entrenar el clasificador Naive Bayes
    gnb_cancer_4 = GaussianNB()
    gnb_cancer_4.fit(X_train_scaled_cancer_4, y_train_cancer_4)

    # Realizar predicciones
    y_pred_cancer_gnb_4 = gnb_cancer_4.predict(X_test_scaled_cancer_4)

    # Evaluar el modelo
    display(Markdown(f"------> **Accuracy Naive Bayes Gaussiano (GNB) ({random_state})**: {accuracy_score(y_test_cancer_4, y_pred_cancer_gnb_4):.4f}"))

    # ======================================================================================================================================
    #
    # MODELADO CON MLP
    #

    mlp_cancer_4 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
    mlp_cancer_4.fit(X_train_scaled_cancer_4, y_train_cancer_4)

    # Realizar predicciones
    y_pred_cancer_mlp_4 = mlp_cancer_4.predict(X_test_scaled_cancer_4)

    # Evaluar el modelo
    display(Markdown(f"------> **Accuracy Perceptrón Multicapa (MLP) ({random_state})**: {accuracy_score(y_test_cancer_4, y_pred_cancer_mlp_4):.4f}"))

# ======================================================================================================================================
#
# COMPARACIÓN DE MODELOS
#

display(Markdown("### Comparación de Modelos para distintos `random_state` en el Dataset Cáncer de Mama"))
display(Markdown("Según la `accurancy` observada para 6 `random_state` diferentes, no hay grandes variaciones. A priori esto podría indicar que ese parámetro de la función `train_test_split` no afecta tanto a los modelos."))

------> **Accuracy K-NN (42)**: 0.9561

------> **Accuracy Naive Bayes Gaussiano (GNB) (42)**: 0.9298

------> **Accuracy Perceptrón Multicapa (MLP) (42)**: 0.9649

------> **Accuracy K-NN (84)**: 0.9649

------> **Accuracy Naive Bayes Gaussiano (GNB) (84)**: 0.9386

...

------> **Accuracy K-NN (1344)**: 0.9649

------> **Accuracy Naive Bayes Gaussiano (GNB) (1344)**: 0.9561

------> **Accuracy Perceptrón Multicapa (MLP) (1344)**: 0.9561

### Comparación de Modelos para distintos `random_state` en el Dataset Cáncer de Mama

Según la `accurancy` observada para 6 `random_state` diferentes, no hay grandes variaciones. A priori esto podría indicar que ese parámetro de la función `train_test_split` no afecta tanto a los modelos.

#### 5. **Comparar con Regresión Logística:** Como un modelo de base adicional, importa `LogisticRegression` de `sklearn.linear_model`. Entrénalo y evalúalo en el dataset Wine. ¿Cómo se compara su rendimiento con los tres modelos de esta clase?

`Respuesta`:

In [103]:
# ======================================================================================================================================
#
# PREPARACIÓN DEL ENTORNO
#
import pandas as pd
import numpy as np
from IPython.display import display, Markdown
from sklearn.linear_model import LogisticRegression

# ======================================================================================================================================
#
# CARGA Y EXPLORACIÓN DE DATOS
#
from sklearn.datasets import load_wine

# Cargar
data_vinos_5 = load_wine(as_frame=True)
df_vinos_5 = data_vinos_5.frame

# Características y objetivo
X_vinos_5 = df_vinos_5[data_vinos_5.feature_names]
y_vinos_5 = df_vinos_5['target']

# Clases
display(Markdown(f"**Nombres de clases**: {data_vinos_5.target_names}"))

# ======================================================================================================================================
#
# PREPARACIÓN DE DATOS
#

# Separación
X_train_vinos_5, X_test_vinos_5, y_train_vinos_5, y_test_vinos_5 = train_test_split(X_vinos_5, y_vinos_5, test_size=0.2, random_state=42, stratify=y_vinos_5)

# Nuevo escalado, en este caso, para el entrenamiento y prueba
scaler_vinos_5 = StandardScaler()
X_train_scaled_vinos_5 = scaler_vinos_5.fit_transform(X_train_vinos_5)
X_test_scaled_vinos_5 = scaler_vinos_5.transform(X_test_vinos_5)

# ======================================================================================================================================
#
# MODELADO CON K-NN
#

start_vinos_5_knn = time.time()

# Inicializar y entrenar el clasificador k-NN
knn_vinos_5 = KNeighborsClassifier(n_neighbors=5)
knn_vinos_5.fit(X_train_scaled_vinos_5, y_train_vinos_5)

# Realizar predicciones
y_pred_vinos_5_knn = knn_vinos_5.predict(X_test_scaled_vinos_5)

# Evaluar el modelo
display(Markdown("------ **Resultados de k-NN (k=5) para Vinos** ------"))
display(Markdown(f"**Accuracy**: {accuracy_score(y_test_vinos_5, y_pred_vinos_5_knn):.4f}"))
display(Markdown("\n**Matriz de Confusión:**"))
print(confusion_matrix(y_test_vinos_5, y_pred_vinos_5_knn))
display(Markdown("\n**Reporte de Clasificación:**"))
print(classification_report(y_test_vinos_5, y_pred_vinos_5_knn))
end_vinos_5_knn = time.time()
display(Markdown(f"**{end_vinos_5_knn - start_vinos_5_knn:.3f}** segundos para entrenar el modelo k-NN para Vinos"))
display(Markdown("\n------------------------------------------------------------\n"))

# ======================================================================================================================================
#
# MODELADO CON NAIVE BAYES
#

start_vinos_5_gnb = time.time()

# Inicializar y entrenar el clasificador Naive Bayes
gnb_vinos_5 = GaussianNB()
gnb_vinos_5.fit(X_train_scaled_vinos_5, y_train_vinos_5)

# Realizar predicciones
y_pred_vinos_5_gnb = gnb_vinos_5.predict(X_test_scaled_vinos_5)

# Evaluar el modelo
display(Markdown("------ **Resultados de Naive Bayes Gaussiano para Vinos** ------"))
display(Markdown(f"**Accuracy**: {accuracy_score(y_test_vinos_5, y_pred_vinos_5_gnb):.4f}"))
display(Markdown("\n**Matriz de Confusión:**"))
print(confusion_matrix(y_test_vinos_5, y_pred_vinos_5_gnb))
display(Markdown("\n**Reporte de Clasificación:**"))
print(classification_report(y_test_vinos_5, y_pred_vinos_5_gnb))
end_vinos_5_gnb = time.time()
display(Markdown(f"**{end_vinos_5_gnb - start_vinos_5_gnb:.3f}** segundos para entrenar el modelo Naive Bayes para Vinos"))
display(Markdown("\n------------------------------------------------------------\n"))

# ======================================================================================================================================
#
# MODELADO CON MLP
#

start_vinos_5_mlp = time.time()

# Inicializar y entrenar el clasificador MLP
mlp_vinos_5 = MLPClassifier(hidden_layer_sizes=(100,), max_iter=1000, random_state=42)
mlp_vinos_5.fit(X_train_scaled_vinos_5, y_train_vinos_5)

# Realizar predicciones
y_pred_vinos_5_mlp = mlp_vinos_5.predict(X_test_scaled_vinos_5)

# Evaluar el modelo
display(Markdown("------ **Resultados del Perceptrón Multicapa (MLP) para Vinos** ------"))
display(Markdown(f"**Accuracy**: {accuracy_score(y_test_vinos_5, y_pred_vinos_5_mlp):.4f}"))
display(Markdown("\n**Matriz de Confusión:**"))
print(confusion_matrix(y_test_vinos_5, y_pred_vinos_5_mlp))
display(Markdown("\n**Reporte de Clasificación:**"))
print(classification_report(y_test_vinos_5, y_pred_vinos_5_mlp))
end_vinos_5_mlp = time.time()
display(Markdown(f"**{end_vinos_5_mlp - start_vinos_5_mlp:.3f}** segundos para entrenar el modelo MLP para Vinos"))
display(Markdown("\n------------------------------------------------------------\n"))

# ======================================================================================================================================
#
# MODELADO CON REGRESIÓN LOGÍSTICA
#

start_vinos_5_lr = time.time()

# Inicializar y entrenar el clasificador de Regresión Logística
lr_vinos_5 = LogisticRegression(max_iter=1000, random_state=42)
lr_vinos_5.fit(X_train_scaled_vinos_5, y_train_vinos_5)

# Realizar predicciones
y_pred_vinos_5_lr = lr_vinos_5.predict(X_test_scaled_vinos_5)

# Evaluar el modelo
display(Markdown("------ **Resultados de Regresión Logística para Vinos** ------"))
display(Markdown(f"**Accuracy**: {accuracy_score(y_test_vinos_5, y_pred_vinos_5_lr):.4f}"))
display(Markdown("\n**Matriz de Confusión:**"))
print(confusion_matrix(y_test_vinos_5, y_pred_vinos_5_lr))
display(Markdown("\n**Reporte de Clasificación:**"))
print(classification_report(y_test_vinos_5, y_pred_vinos_5_lr))
end_vinos_5_lr = time.time()
display(Markdown(f"**{end_vinos_5_lr - start_vinos_5_lr:.3f}** segundos para entrenar el modelo de Regresión Logística para Vinos"))
display(Markdown("\n------------------------------------------------------------\n"))

# ======================================================================================================================================
#
# COMPARACIÓN DE MODELOS
#

display(Markdown("### Comparación de Modelos para el Dataset Vinos"))
display(Markdown("#### **Precisión**"))
display(Markdown("Según `accurancy`, la regresión logística es tan precisa como los 3 modelos supervisados estudiados en esta práctica. "))
display(Markdown("#### **Matriz de confusión**"))
display(Markdown("Si miramos la `matriz de confusión`, tampoco hay demasiadas diferencias con los otros 3 modelos."))
display(Markdown("#### **Reporte de clasificación**"))
display(Markdown("Si miramos el `reporte de clasificación`, tampoco se notan diferencias sustanciales con los otros 3 modelos."))
display(Markdown("#### **Tiempo de entrenamiento**"))
display(Markdown("En cuanto al `tiempo de entrenamiento`, el modelo de regresión logística es más rápido que MLP (por diferencia), pero no que k-NN y GNB (acá no es tanta la diferencia)."))
display(Markdown("#### **Conclusión**:"))
display(Markdown("Para el dataset de vinos, la regresión logística se comporta tan bien como los modelos supervisados estudiados (MLP, GNB y k-NN). Por lo que, es una buena opción a considerar."))

**Nombres de clases**: ['class_0' 'class_1' 'class_2']

------ **Resultados de k-NN (k=5) para Vinos** ------

**Accuracy**: 0.9722


**Matriz de Confusión:**

[[12  0  0]
 [ 0 13  1]
 [ 0  0 10]]


...

Si miramos el `reporte de clasificación`, tampoco se notan diferencias sustanciales con los otros 3 modelos.

#### **Tiempo de entrenamiento**

En cuanto al `tiempo de entrenamiento`, el modelo de regresión logística es más rápido que MLP (por diferencia), pero no que k-NN y GNB (acá no es tanta la diferencia).

#### **Conclusión**:

Para el dataset de vinos, la regresión logística se comporta tan bien como los modelos supervisados estudiados (MLP, GNB y k-NN). Por lo que, es una buena opción a considerar.